In [423]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/corona-virus-report/covid_19_clean_complete.csv
/kaggle/input/corona-virus-report/country_wise_latest.csv
/kaggle/input/corona-virus-report/day_wise.csv
/kaggle/input/corona-virus-report/usa_county_wise.csv
/kaggle/input/corona-virus-report/worldometer_data.csv
/kaggle/input/corona-virus-report/full_grouped.csv


In [424]:
#Item 1

df_1 = pd.read_csv('/kaggle/input/corona-virus-report/covid_19_clean_complete.csv') #adicionando o dataset covid 19 clean complete
df_1 = df_1.rename(columns={'Province/State': 'Province_State'}) #A barra estava dando problema no código

df_1 = df_1.rename(columns={'Country/Region': 'Country_Region'}) #A barra estava dando problema no código


n_linhas , n_colunas = df_1.shape

print(f'Número de linhas: ', n_linhas , '\n')
print(f'Número de colunas:', n_colunas , '\n')



print('Colunas:',' , '.join(df_1.columns) ,'\n')



print(df_1.dtypes ,'\n')

df_1['Date']  = pd.to_datetime(df_1['Date'])

print('O tipo de dado das datas é:' , df_1.Date.dtype ,'\n')

print('Dados estatísticos sobre os casos:', '\n')

print(df_1[['Confirmed','Deaths','Recovered','Active']].describe(), '\n')

colunas_com_nulos = df_1.columns[df_1.isnull().any()].tolist()

print("Colunas com valores nulos:" , colunas_com_nulos)
























Número de linhas:  49068 

Número de colunas: 10 

Colunas: Province_State , Country_Region , Lat , Long , Date , Confirmed , Deaths , Recovered , Active , WHO Region 

Province_State     object
Country_Region     object
Lat               float64
Long              float64
Date               object
Confirmed           int64
Deaths              int64
Recovered           int64
Active              int64
WHO Region         object
dtype: object 

O tipo de dado das datas é: datetime64[ns] 

Dados estatísticos sobre os casos: 

          Confirmed         Deaths     Recovered        Active
count  4.906800e+04   49068.000000  4.906800e+04  4.906800e+04
mean   1.688490e+04     884.179160  7.915713e+03  8.085012e+03
std    1.273002e+05    6313.584411  5.480092e+04  7.625890e+04
min    0.000000e+00       0.000000  0.000000e+00 -1.400000e+01
25%    4.000000e+00       0.000000  0.000000e+00  0.000000e+00
50%    1.680000e+02       2.000000  2.900000e+01  2.600000e+01
75%    1.518250e+03      30.0000

In [425]:
#Item 2



china = 'China'
df_1_china = df_1[df_1["Country_Region"] == china]

df_1_provincias = df_1_china.groupby("Province_State")["Confirmed"].sum()

df_1_provincias_ordenado = df_1_provincias.sort_values(ascending=False)

df_1_provincias_ordenado.head(5)

new_df_1 = df_1.loc[df_1.Province_State.isin(['Hubei','Guangdong','Henan','Zhejiang','Hunan'])]

df_reduzido = new_df_1.drop(columns=['Lat' , 'Long' , 'Date' , 'WHO Region'])

df_reduzido = df_reduzido.reset_index(drop=True) #Faz com que comece em 0 o novo dataframe


df_reduzido






,Province_State,Country_Region,Confirmed,Deaths,Recovered,Active
0,Guangdong,China,26,0,0,26
1,Henan,China,5,0,0,5
2,Hubei,China,444,17,28,399
3,Hunan,China,4,0,0,4
4,Zhejiang,China,10,0,0,10
...,...,...,...,...,...,...
935,Guangdong,China,1672,8,1645,19
936,Henan,China,1276,22,0,1254
937,Hubei,China,68135,4512,63623,0
938,Hunan,China,1019,4,1015,0


In [426]:
#Item 3

df_1['Country_Region'] = df_1.apply(
    lambda row: f"{row['Country_Region']}_{row['Province_State']}" if pd.notna(row['Province_State']) else row['Country_Region'], 
    axis=1
)


df_1 = df_1.drop(columns=['Province_State'])



df_1.sample(5)






,Country_Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
38717,Czechia,49.817500,15.473000,2020-06-18,10280,334,7440,2506,Europe
47204,Uzbekistan,41.377491,64.585262,2020-07-20,17149,90,9387,7672,Europe
44477,France_Guadeloupe,16.265000,-61.551000,2020-07-10,190,14,157,19,Europe
8105,Australia_Victoria,-37.813600,144.963100,2020-02-22,4,0,4,0,Western Pacific
23252,Belgium,50.833300,4.469936,2020-04-20,39983,5828,8895,25260,Europe


In [427]:
#Item 4

df_2 = pd.read_csv('/kaggle/input/corona-virus-report/worldometer_data.csv') #adicionando o dataset worldometer data
df_2 = df_2.rename(columns={'Country/Region': 'Country_Region'})

df_2new  = df_2[['Country_Region' , 'Continent' , 'Population']]

df_1agrupado = df_1.select_dtypes(include='number').groupby(df_1['Country_Region']).sum().reset_index() #Desconsiderar as Datas.

df_mesclado = pd.merge(df_1agrupado, df_2new, on='Country_Region', how='inner')

df_mesclado1 = df_mesclado[['Continent' , 'Population' , 'Deaths']]

df_mesclado1 = df_mesclado1.groupby(df_mesclado1['Continent']).sum().reset_index()

df_mesclado1['Deaths per Million'] = (df_mesclado1['Deaths'] / df_mesclado1['Population'])*(10**6)

df_mesclado1.sort_values(by='Deaths per Million', ascending=False )




,Continent,Population,Deaths,Deaths per Million
3,Europe,6.794130e+08,14680439,21607.531868
5,South America,4.308076e+08,5681838,13188.806632
4,North America,2.190365e+08,1966477,8977.850396
1,Asia,3.020510e+09,3882965,1285.533099
0,Africa,1.215738e+09,682067,561.031368
2,Australia/Oceania,1.486220e+07,2183,146.882656
